In [4]:
"""
solve_and_submit_turnstile.py

Usage:
  pip install requests beautifulsoup4
  python3 solve_and_submit_turnstile.py

Remplis API_KEY et PAGE_URL dans la section CONFIGURATION ci-dessous.
"""

import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

# ---------------- CONFIGURATION ----------------
API_KEY = "e8afb42ed3de82c60392cfea55ecf555"                         # <-- remplace par ta clé 2captcha
PAGE_URL = "https://dl-protect.link/9a803ade?fn=TGVzIDQgRmFudGFzdGlxdWVzIDogUHJlbWllcnMgcGFzIFtXRUJSSVAgNEtdIC0gTVVMVEkgKFRSVUVGUkVOQ0gp&rl=a2" # <-- remplace par l'URL de la page contenant le formulaire
MAX_WAIT_SECONDS = 200                              # timeout max pour la résolution
POLL_INTERVAL = 5                                   # intervalle de polling (s)
# ------------------------------------------------

HEADERS = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120 Safari/537.36"
}

def fetch_page(session: requests.Session, url: str):
    r = session.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return r.text, r.url  # renvoie HTML et URL finale (redirects possibles)

def find_form_and_sitekey(html: str):
    soup = BeautifulSoup(html, "html.parser")

    # Cherche d'abord un div Turnstile
    div_turn = soup.find("div", class_="cf-turnstile")
    if div_turn and div_turn.get("data-sitekey"):
        return soup, div_turn.get("data-sitekey"), div_turn

    # Sinon recherche reCAPTCHA / hCaptcha (fallback)
    rec = soup.find(attrs={"data-sitekey": True})
    if rec:
        sk = rec.get("data-sitekey")
        return soup, sk, rec

    # Si pas trouvé
    return soup, None, None

def extract_form(soup: BeautifulSoup):
    # On récupère le premier formulaire 'visible' central ; tu peux adapter la logique si plusieurs forms existent
    form = soup.find("form")
    if not form:
        raise RuntimeError("Aucun <form> trouvé sur la page.")
    # action (relatif possible) + méthode (POST par défaut)
    action = form.get("action") or ""
    method = (form.get("method") or "GET").upper()
    # récupère tous les inputs (name -> value)
    data = {}
    for inp in form.find_all("input"):
        name = inp.get("name")
        if not name:
            continue
        # prend la value (vide si absent)
        value = inp.get("value", "")
        data[name] = value
    return form, action, method, data

def submit_to_2captcha_turnstile(api_key: str, sitekey: str, pageurl: str):
    in_url = "https://2captcha.com/in.php"
    params = {
        "key": api_key,
        "method": "turnstile",
        "pageurl": pageurl,
        "sitekey": sitekey,
        "json": 1
    }
    resp = requests.get(in_url, params=params, timeout=30)
    j = resp.json()
    if j.get("status") != 1:
        raise RuntimeError(f"Erreur soumission 2Captcha: {j}")
    return j["request"]  # id

def poll_2captcha_result(api_key: str, captcha_id: str, max_wait: int = MAX_WAIT_SECONDS, poll_interval: int = POLL_INTERVAL):
    res_url = "https://2captcha.com/res.php"
    waited = 0
    while waited < max_wait:
        time.sleep(poll_interval)
        waited += poll_interval
        r = requests.get(res_url, params={"key": api_key, "action": "get", "id": captcha_id, "json": 1}, timeout=30)
        j = r.json()
        if j.get("status") == 1:
            return j["request"]
        if j.get("request") and j.get("request") != "CAPCHA_NOT_READY":
            raise RuntimeError(f"Erreur 2Captcha pendant polling: {j}")
    raise TimeoutError("Timeout waiting for 2Captcha result")

def main():
    session = requests.Session()
    session.headers.update(HEADERS)

    print(f"[+] Téléchargement de la page : {PAGE_URL}")
    html, final_url = fetch_page(session, PAGE_URL)
    soup, sitekey, sitekey_node = find_form_and_sitekey(html)
    if not sitekey:
        raise RuntimeError("Impossible de trouver de sitekey Turnstile / reCAPTCHA sur la page.")

    print(f"[+] Sitekey trouvé : {sitekey}")
    form, action, method, form_data = extract_form(soup)
    print(f"[+] Form trouvé : method={method}, action='{action}'. Champs initiaux: {list(form_data.keys())}")

    # Normaliser action en URL absolue
    action_url = urljoin(final_url, action) if action else final_url
    print(f"[+] URL d'action du formulaire : {action_url}")

    # Soumettre le captcha à 2captcha
    print("[+] Soumission du Turnstile à 2Captcha...")
    captcha_id = submit_to_2captcha_turnstile(API_KEY, sitekey, final_url)
    print(f"[+] ID 2Captcha : {captcha_id}")

    print("[+] Polling pour le token...")
    token = poll_2captcha_result(API_KEY, captcha_id, max_wait=MAX_WAIT_SECONDS, poll_interval=POLL_INTERVAL)
    print(f"[+] Token reçu (début) : {token[:60]}...")

    # Nom du champ Turnstile
    # Cloudflare utilise 'cf-turnstile-response' (mais on essaie aussi g-recaptcha-response si absent)
    field_name = None
    # si présent dans le form initial on réutilise son nom
    if "cf-turnstile-response" in form_data:
        field_name = "cf-turnstile-response"
    elif "g-recaptcha-response" in form_data:
        field_name = "g-recaptcha-response"
    else:
        # par défaut pour turnstile
        field_name = "cf-turnstile-response"

    # Injecter / mettre à jour la valeur dans le payload
    form_data[field_name] = token

    print(f"[+] Envoi POST vers {action_url} avec le jeton injecté sous '{field_name}'")
    # POST avec cookies et headers de session
    if method == "POST":
        resp = session.post(action_url, data=form_data, allow_redirects=True, timeout=60)
    else:
        resp = session.get(action_url, params=form_data, allow_redirects=True, timeout=60)
        
    print(f"[+] Statut réponse : {resp.status_code}")
    # Affiche un extrait de la réponse pour vérification
    text = resp.text or ""
    print("----- Début réponse (500 char) -----")
    print(text[:2000])
    print("----- Fin extrait -----")

    # Optionnel : vérifie si le serveur a accepté le formulaire (heuristique simple)
    if resp.status_code in (200, 302):
        print("[+] Requête envoyée — vérifie côté serveur si l'action a été acceptée.")
    else:
        print("[!] Réponse inattendue, inspecte le HTML retourné pour trouver pourquoi.")
    return text



In [5]:
try:
    text = main()
except Exception as e:
    print("Erreur:", e)

[+] Téléchargement de la page : https://dl-protect.link/9a803ade?fn=TGVzIDQgRmFudGFzdGlxdWVzIDogUHJlbWllcnMgcGFzIFtXRUJSSVAgNEtdIC0gTVVMVEkgKFRSVUVGUkVOQ0gp&rl=a2
[+] Sitekey trouvé : 0x4AAAAAAABKK-fmValRCMjW
[+] Form trouvé : method=POST, action=''. Champs initiaux: ['subform']
[+] URL d'action du formulaire : https://dl-protect.link/9a803ade?fn=TGVzIDQgRmFudGFzdGlxdWVzIDogUHJlbWllcnMgcGFzIFtXRUJSSVAgNEtdIC0gTVVMVEkgKFRSVUVGUkVOQ0gp&rl=a2
[+] Soumission du Turnstile à 2Captcha...
[+] ID 2Captcha : 80744005182
[+] Polling pour le token...
[+] Token reçu (début) : 0.KKhRoVn3h15xphLy8C2yENxHqZHDIK8IiPhOqQe4VXBdmmr6I6RzKME46t...
[+] Envoi POST vers https://dl-protect.link/9a803ade?fn=TGVzIDQgRmFudGFzdGlxdWVzIDogUHJlbWllcnMgcGFzIFtXRUJSSVAgNEtdIC0gTVVMVEkgKFRSVUVGUkVOQ0gp&rl=a2 avec le jeton injecté sous 'cf-turnstile-response'
[+] Statut réponse : 200
----- Début réponse (500 char) -----
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <title>Protect your links</title>
	
	<lin

In [10]:
print(text[2000:4000])

e publicité.</p></b></center></div>
<div><center><b><p>Nous vous conseillons encore une fois de <a href="https://one.one.one.one/fr-FR/dns/" target="_blank">CHANGER DE DNS iCi</a> pour éviter les blocages de FAI et avoir une connexion rapide et sécurisée.</p></b></center></div>        <div id="protected-container" class="container">

			<div class="row">
				<div class="col-sm-12">
					<center>
												<!--<a style="cursor:pointer;" href="https://galaxyearningsfly.com/trck/eclick/3750a4571aa80b064d61a4bc92ad6e72"><img src="/images3/afl5.png" onmouseout="this.src='/images3/afl5.png'" onmouseover="this.src='/images3/afl6.png'"/></a><br/><br/>
						--><span class="amigo" ><a style="cursor:pointer;" href="/getfile?fn=aHR0cHM6Ly9nYWxheHllYXJuaW5nc2ZseS5jb20vdHJjay9lY2xpY2svMzc1MGE0NTcxYWE4MGIwNjRkNjFhNGJjOTJhZDZlNzI=&"><img src="/images3/afl5.png" onmouseout="this.src='/images3/afl5.png'" onmouseover="this.src='/images3/afl6.png'"/></a><br/><br/></span>
					</center>
				</div>
			</

In [13]:
#!/usr/bin/env python3
"""
solve_and_extract_link.py

Même principe que précédemment, mais main() renvoie (et affiche) le href du premier
élément <a> dont l'attribut rel contient 'external' et 'nofollow'.

Usage:
  pip install requests beautifulsoup4
  python3 solve_and_extract_link.py
"""

import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# ---------------- CONFIGURATION ----------------
API_KEY = "e8afb42ed3de82c60392cfea55ecf555"                         # <-- remplace par ta clé 2captcha
PAGE_URL = "https://dl-protect.link/9a803ade?fn=TGVzIDQgRmFudGFzdGlxdWVzIDogUHJlbWllcnMgcGFzIFtXRUJSSVAgNEtdIC0gTVVMVEkgKFRSVUVGUkVOQ0gp&rl=a2" # <-- remplace par l'URL de la page contenant le formulaire
MAX_WAIT_SECONDS = 200                              # timeout max pour la résolution
POLL_INTERVAL = 5                                   # intervalle de polling (s)
# ------------------------------------------------

HEADERS = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120 Safari/537.36"
}

def fetch_page(session: requests.Session, url: str):
    r = session.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return r.text, r.url  # renvoie HTML et URL finale (redirects possibles)

def find_form_and_sitekey(html: str):
    soup = BeautifulSoup(html, "html.parser")

    # Cherche d'abord un div Turnstile
    div_turn = soup.find("div", class_="cf-turnstile")
    if div_turn and div_turn.get("data-sitekey"):
        return soup, div_turn.get("data-sitekey"), div_turn

    # Sinon recherche reCAPTCHA / hCaptcha (fallback)
    rec = soup.find(attrs={"data-sitekey": True})
    if rec:
        sk = rec.get("data-sitekey")
        return soup, sk, rec

    # Si pas trouvé
    return soup, None, None

def extract_form(soup: BeautifulSoup):
    form = soup.find("form")
    if not form:
        raise RuntimeError("Aucun <form> trouvé sur la page.")
    action = form.get("action") or ""
    method = (form.get("method") or "GET").upper()
    data = {}
    for inp in form.find_all("input"):
        name = inp.get("name")
        if not name:
            continue
        value = inp.get("value", "")
        data[name] = value
    return form, action, method, data

def submit_to_2captcha_turnstile(api_key: str, sitekey: str, pageurl: str):
    in_url = "https://2captcha.com/in.php"
    params = {
        "key": api_key,
        "method": "turnstile",
        "pageurl": pageurl,
        "sitekey": sitekey,
        "json": 1
    }
    resp = requests.get(in_url, params=params, timeout=30)
    j = resp.json()
    if j.get("status") != 1:
        raise RuntimeError(f"Erreur soumission 2Captcha: {j}")
    return j["request"]  # id

def poll_2captcha_result(api_key: str, captcha_id: str, max_wait: int = MAX_WAIT_SECONDS, poll_interval: int = POLL_INTERVAL):
    res_url = "https://2captcha.com/res.php"
    waited = 0
    while waited < max_wait:
        time.sleep(poll_interval)
        waited += poll_interval
        r = requests.get(res_url, params={"key": api_key, "action": "get", "id": captcha_id, "json": 1}, timeout=30)
        j = r.json()
        if j.get("status") == 1:
            return j["request"]
        if j.get("request") and j.get("request") != "CAPCHA_NOT_READY":
            raise RuntimeError(f"Erreur 2Captcha pendant polling: {j}")
    raise TimeoutError("Timeout waiting for 2Captcha result")

def find_external_nofollow_href(html: str, base_url: str):
    """
    Retourne le href du premier <a> dont rel contient 'external' et 'nofollow'.
    Si href est relatif, le convertit en URL absolue via base_url.
    """
    soup = BeautifulSoup(html, "html.parser")
    # Parcours tous les <a> et vérifie l'attribut rel
    for a in soup.find_all("a", href=True):
        rel = a.get("rel")
        if not rel:
            # parfois rel est une string au lieu d'une liste
            rel_attr = a.get("rel")
        else:
            rel_attr = rel
        # normaliser en string pour test (ex: ['external','nofollow'] ou "external nofollow")
        if isinstance(rel_attr, (list, tuple)):
            rel_str = " ".join(rel_attr).lower()
        else:
            rel_str = str(rel_attr).lower()
        # vérifier la présence des deux tokens
        if "external" in rel_str and "nofollow" in rel_str:
            href = a["href"]
            return urljoin(base_url, href)
    return None

def main():
    session = requests.Session()
    session.headers.update(HEADERS)

    print(f"[+] Téléchargement de la page : {PAGE_URL}")
    html, final_url = fetch_page(session, PAGE_URL)

    soup, sitekey, _ = find_form_and_sitekey(html)
    if not sitekey:
        raise RuntimeError("Impossible de trouver de sitekey Turnstile / reCAPTCHA sur la page.")

    print(f"[+] Sitekey trouvé : {sitekey}")
    form, action, method, form_data = extract_form(soup)
    action_url = urljoin(final_url, action) if action else final_url
    print(f"[+] URL d'action du formulaire : {action_url}")

    # Soumettre le captcha à 2captcha
    print("[+] Soumission du Turnstile à 2Captcha...")
    captcha_id = submit_to_2captcha_turnstile(API_KEY, sitekey, final_url)
    print(f"[+] ID 2Captcha : {captcha_id}")

    print("[+] Polling pour le token...")
    token = poll_2captcha_result(API_KEY, captcha_id, max_wait=MAX_WAIT_SECONDS, poll_interval=POLL_INTERVAL)
    print(f"[+] Token reçu (début) : {token[:60]}...")

    # Nom du champ Turnstile
    field_name = "cf-turnstile-response"
    if "cf-turnstile-response" in form_data:
        field_name = "cf-turnstile-response"
    elif "g-recaptcha-response" in form_data:
        field_name = "g-recaptcha-response"

    form_data[field_name] = token

    print(f"[+] Envoi {method} vers {action_url} avec le jeton injecté sous '{field_name}'")
    if method == "POST":
        resp = session.post(action_url, data=form_data, allow_redirects=True, timeout=60)
    else:
        resp = session.get(action_url, params=form_data, allow_redirects=True, timeout=60)

    print(f"[+] Statut réponse : {resp.status_code}")

    # Cherche le lien <a rel="external nofollow"> dans la réponse finale
    href = find_external_nofollow_href(resp.text, resp.url)
    if href:
        print("[+] Lien trouvé :", href)
    else:
        print("[!] Aucun lien <a rel='external nofollow'> trouvé dans la réponse.")

    # retourne le href (ou None)
    return href




In [14]:
if __name__ == "__main__":
    try:
        result = main()
        print("\n[=] Résultat renvoyé par main():", result)
    except Exception as e:
        print("Erreur:", e)

[+] Téléchargement de la page : https://dl-protect.link/9a803ade?fn=TGVzIDQgRmFudGFzdGlxdWVzIDogUHJlbWllcnMgcGFzIFtXRUJSSVAgNEtdIC0gTVVMVEkgKFRSVUVGUkVOQ0gp&rl=a2
[+] Sitekey trouvé : 0x4AAAAAAABKK-fmValRCMjW
[+] URL d'action du formulaire : https://dl-protect.link/9a803ade?fn=TGVzIDQgRmFudGFzdGlxdWVzIDogUHJlbWllcnMgcGFzIFtXRUJSSVAgNEtdIC0gTVVMVEkgKFRSVUVGUkVOQ0gp&rl=a2
[+] Soumission du Turnstile à 2Captcha...
[+] ID 2Captcha : 80744048933
[+] Polling pour le token...
[+] Token reçu (début) : 0.w7RsPpVLTprYy7hgMMPMiDNxsspl_oekFcSWKtZuEbujMNwP2ktmvn925v...
[+] Envoi POST vers https://dl-protect.link/9a803ade?fn=TGVzIDQgRmFudGFzdGlxdWVzIDogUHJlbWllcnMgcGFzIFtXRUJSSVAgNEtdIC0gTVVMVEkgKFRSVUVGUkVOQ0gp&rl=a2 avec le jeton injecté sous 'cf-turnstile-response'
[+] Statut réponse : 200
[+] Lien trouvé : https://1fichier.com/?a7p2gep44kiisgvk54ov&af=3108529

[=] Résultat renvoyé par main(): https://1fichier.com/?a7p2gep44kiisgvk54ov&af=3108529


In [ ]:
import requests
from urllib.parse import urlparse

DEFAULT_TIMEOUT = 15

def is_link_alive(url, session=None, timeout=DEFAULT_TIMEOUT, max_head_size=1024):
    """
    Teste si une URL de fichier est "vivante".
    Retourne un tuple (alive:bool, info:dict)
    info contient: status_code, reason, content_type, content_length, note
    """
    s = session or requests.Session()
    s.headers.update({
        "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120 Safari/537.36"
    })

    info = {"status_code": None, "reason": None, "content_type": None, "content_length": None, "note": None}
    try:
        # 1) quick HEAD
        resp = s.head(url, allow_redirects=False, timeout=timeout)
        info["status_code"] = resp.status_code
        info["reason"] = resp.reason
        info["content_type"] = resp.headers.get("Content-Type")
        info["content_length"] = resp.headers.get("Content-Length")
        # If HEAD gives a clear positive:
        if 200 <= resp.status_code < 300:
            # If it's binary (non-html) and has length -> very likely alive
            ct = info["content_type"] or ""
            if "text/html" not in ct.lower():
                info["note"] = "HEAD OK and non-HTML content-type"
                return True, info
            # if HTML returned on HEAD, we need to inspect content (some hosts respond HTML even for files)
        elif resp.status_code in (403, 401):
            # Some hosts block HEAD; fallthrough to GET test
            info["note"] = f"HEAD returned {resp.status_code}; trying GET"
        elif resp.status_code in (404, 410):
            info["note"] = "Not found according to HEAD"
            return False, info
        elif 300<= resp.status_code <400 :
            parsed = urlparse(url)
            hostname_base = parsed.hostname
            parsed = urlparse(resp.headers["Location"])
            hostname_redirection = parsed.hostname
            if hostname_redirection == None:
                return is_link_alive("https://"+hostname_base+resp.headers["Location"])
            print(hostname_base)
            print(hostname_redirection)
            if hostname_base != hostname_redirection:
                return is_link_alive(resp.headers["Location"])
            info["note"] = "Redirection pas normal"
            return False,info
        else:
            # other 3xx / 4xx / 5xx -> try GET for safety
            info["note"] = f"HEAD returned {resp.status_code}; trying GET"
    except requests.RequestException as e:
        info["note"] = f"HEAD failed: {e}; trying GET"

    # 2) Fallback: small GET to check body (use Range to avoid big download)
    try:
        headers = {"Range": f"bytes=0-{max_head_size-1}"}
        resp = s.get(url, allow_redirects=True, timeout=timeout, headers=headers, stream=True)
        info["status_code"] = resp.status_code
        info["reason"] = resp.reason
        info["content_type"] = resp.headers.get("Content-Type")
        info["content_length"] = resp.headers.get("Content-Length")
        body_snippet = resp.content[:max_head_size].decode('utf-8', errors='ignore').lower()

        # common patterns indicating a dead link (customize per host)
        dead_signatures = [
            "file not found", "not found", "404", "no such file", "the file was removed",
            "this file has been deleted", "error 404", "page not found", "no such user",
        ]
        # host specific quick checks (example: 1fichier returns HTML error message)
        host = urlparse(url).netloc.lower()
        if resp.status_code in (200, 206):  # 206 if range supported
            # if content-type is HTML and body contains error phrases -> dead
            ct = (info["content_type"] or "").lower()
            if "text/html" in ct:
                for sig in dead_signatures:
                    if sig in body_snippet:
                        info["note"] = f"HTML page contains error signature: '{sig}'"
                        return False, info
                # otherwise could still be a valid download page (some hosts require JS)

                if  "searching for the file" in resp.text.lower():
                    info["note"] = "turbobit dont find"
                    return False, info
                info["note"] = "GET returned HTML but no obvious error signature"
                return True, info
            else:
                info["note"] = "GET returned non-HTML (likely file) => alive"
                return True, info
        elif resp.status_code in (403, 401):
            info["note"] = "Access denied (403/401)"
            return False, info
        elif resp.status_code in (404, 410):
            info["note"] = "Not found (404/410)"
            return False, info
        else:
            info["note"] = f"GET returned {resp.status_code}"
            # consider dead for 5xx or unknown codes
            return False, info
    except requests.RequestException as e:
        info["note"] = f"GET failed: {e}"
        return False, info


In [69]:
# availaible_link_1fichier = "https://1fichier.com/?a7p2gep44kiisgvk54ov&af=3108529"
# dead_link_1fichier = "https://1fichier.com/?vdlmejjdib18itr6yz8y&af=3601079"
# rep = is_link_alive(availaible_link_1fichier)
# if rep[0] != True:
#     print("availaible_link_1fichier " +str(rep))

# rep = is_link_alive(dead_link_1fichier)
# print(rep)
# if rep[0] != False:
#     print("dead_link_1fichier"+str(rep))

# dead_link_dailyupload = "https://dailyuploads.net/3mhdgt1nu929"
# availaible_link_dailyupload = "https://dailyuploads.net/72zb0us8oav0"

# rep = is_link_alive(availaible_link_dailyupload)
# if rep [0] != True:
#     print("availaible_link_dailyupload"+str(rep))

# rep = is_link_alive(dead_link_dailyupload)
# if rep [0] != False:
#     print("dead_link_dailyupload"+str(rep))
# dead_link_rapidagtor = "https://rapidgator.net/file/b0ccc2ddf97a2a48e236a6bc65912506"
# availaible_link_rapidagtor = "https://rapidgator.net/file/2426c157d2d2099a3126b3102fd77871"

# rep = is_link_alive(availaible_link_rapidagtor)
# if rep [0] != True:
#     print("availaible_link_rapidagtor"+str(rep))

# rep = is_link_alive(dead_link_rapidagtor)
# if rep [0] != False:
#     print("dead_link_rapidagtor"+str(rep))
# availaible_link_uploady = "https://uploady.io/ck2m22gfrojf"
# dead_link_uploady = "https://uploady.io/t7oas9onunp6"

# rep = is_link_alive(availaible_link_uploady)
# if rep [0] != True:
#     print("availaible_link_uploady"+str(rep))

# rep = is_link_alive(dead_link_uploady)
# if rep [0] != False:
#     print("dead_link_uploady"+str(rep))
availaible_link_nitroflare = "https://nitroflare.com/view/4E11E3F2D22E777"
dead_link_nitroflare = "https://nitroflare.com/view/157FC61BCC0BC76"

rep = is_link_alive(availaible_link_nitroflare)
if rep [0] != True:
    print("availaible_link_nitroflare"+str(rep))

rep = is_link_alive(dead_link_nitroflare)
print(rep)
if rep [0] != False:
    print("dead_link_nitroflare"+str(rep))
# availaible_link_turbobit = "https://trbt.cc/be7yjf6wmd1z.html"
# dead_link_turbobit = "https://trbt.cc/g3ur5yoe2rv5.html"

# rep = is_link_alive(availaible_link_turbobit)
# if rep [0] != True:
#     print("availaible_link_turbobit"+str(rep))

# rep = is_link_alive(dead_link_turbobit)
# if rep [0] != False:
#     print("dead_link_turbobit"+str(rep))

(False, {'status_code': 404, 'reason': 'Not Found', 'content_type': 'text/html; charset=UTF-8', 'content_length': None, 'note': 'Not found according to HEAD'})
